<a href="https://colab.research.google.com/github/SUDHARSSHINI/Drone/blob/main/soil_test_report_with_weather_for_crop_prediction_of_csv_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'soil-test-report-with-weather-for-crop-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4930260%2F8299211%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240913%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240913T135728Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D271aeaa15d534ce3a9d4a022b457cb27adb1dc00e7beb85aef10a8c2072e6b2ef2233c55636269f58de95cc6e60451df247ed1a4b94343fabf42474fb33e27eba3a466b98dac5bd835837274f0aaf08e686e05c2ad62df7a38bd4686f1869d8062efa859bdf81a75f8ef4d1f9fb6cac68dbe66a3202aca22084252173f65fd362cd5930d7efe73d3c4e54b9762bbef3c961a62a40656d08f7376174cdadc3c34a14c4e4ca3048ffb7f53f2c8cf52de405bd68199c824e86c7071aba9f6379935ea54ee1c79db7a138c9726f629aec98876d3428fc1fffac8bdf5498f8cfe3240c019a215e5411028b51b549f2cc883c6cba8a078d49568906e4cd379bd32b229'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


Let's start by importing the libraries. We will begin with importing the basic libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Secondly we will import DecisionTreeClassifier, grid search and metrics of the model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import RocCurveDisplay, confusion_matrix

And finaly we will import train test split

In [ ]:
from sklearn.model_selection import train_test_split

# Preprocessing of the data

We will begin with reading the data

In [ ]:
data  = pd.read_csv('/kaggle/input/soil-test-report-with-weather-for-crop-prediction/Crop_recommendation.csv')

We can see that the dataset has 22 labels

In [ ]:
labels = data['label'].unique()
print(labels)

Then we will transform the labels as numbers by replacing the values as index of the arraya

In [ ]:
data['label'] = data['label'].apply(lambda x: np.where(labels == x)[0][0])

After that we will split the data for training and testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['label']), data['label'], test_size=0.2, random_state=42)

Finally we will make GridSearchCV to choose the best parameters for the model

In [ ]:
parameters= {
    'max_depth': [5,10,11,12,13,14,15,None],
    'min_samples_split': [10,20,30,40,50,60,70,80,90,100],
}
cv = GridSearchCV(DecisionTreeClassifier(), parameters,cv=3,n_jobs=7)
cv.fit(x_train, y_train)

print(cv.best_params_)


y_pred = cv.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print('Accuracy: ', accuracy)
print('F1: ', f1)
print('Precision: ', precision)
print('Recall: ', recall)

The results of the model show a high accuracy and an F1 score, which means that it is effectively classified with a high degree of reliability and a balance between accuracy and estimation.